In [ ]:
#import all search functions and dataset

In [7]:
from tkinter import *
from PIL import ImageTk, Image

class search():

    def __init__(self):

        root = Tk()
        root.lift()
        root.attributes("-topmost", True)
        root.geometry("800x800")
        root.title("Amazeng")
        self.root = root

        # Build GUI
        canvas_width = 800
        canvas_height = 800
        canvas = Canvas(root, height=canvas_height, width=canvas_width)
        self.canvas = canvas

        # Insert logo on top
        self.logo_frame = Frame(root, width=canvas_width, height=300)
        self.logo_frame.pack(fill=X,expand=True)
        logo_file = Image.open('main_logo_nobackground_white.png')
        logo_file = logo_file.resize((800,300),Image.ANTIALIAS)
        photo_logo = ImageTk.PhotoImage(logo_file)
        self.logo_label = Label(self.logo_frame, image=photo_logo)
        self.logo_label.pack(filel=X, expand=True)

        # Product Search Box: 
        #clear gap btn entrybox & search button
        #change text button to logo img
        #clear default text when there's user input
        # for more color effects http://effbot.org/tkinterbook/entry.htm
        self.p_search = Entry(root,bd=2,relwidth=0.5,height=20,justify=LEFT,relief=GROOVE)
        self.p_search.insert(END, 'Product...')
        self.p_search.place(relx=0.49, rely=0.4, anchor=CENTER)
        self.p_search.pack(side='top')
        self.p_search.focus_set()
        item = self.p_search.get() #get user input, use as func var
        self.p_button = Button(root, text="Search", width=10)
        self.p_button.pack()

        # Display Keyword with checkboxes
        #align keywords in columns, go to next col if too many
        #delete when clicked x
        #update keyword list
        self.key_frame = Frame(root, width=canvas_width, height=100)
        self.key_frame.pack(fill=X,expand=True)

        # Keyword Search Box
        #same layout adjustment
        #no input until there's a product

        default_state = 'disabled'
        default_color = 'grey'

        self.k_search = Entry(root,bd=2,relwidth=0.5,height=20,state=default_state,bg=default_color,justify=LEFT,relief=GROOVE)
        self.k_search.insert(END, 'Keyword...')
        self.k_search.place(relx=0.3, rely=0.65, anchor=CENTER)
        self.k_search.pack(side='top')
        self.k_search.focus_set()
        if self.p_search.get():
            self.k_search.config(bg='white',state=NORMAL) #enable keyword search after input for product
        cur_key = k_search.get() #get current key
        keys.append(cur_key) #add current key to the list of keywords
        self.cur_key.set(cur_key)
        self.p_button = Button(root, text="Search", width=10)
        self.p_button.pack()
#         if self.k_button.invoke():
                  
        # Display Result
        #adjust size to fit window
        self.result_frame = Frame(root, width=canvas_width, height=300)
        self.result_frame.pack(side='bottom', fill=X, expand=True)
        key = []
        self.scrollbar = Scrollbar(self.result_frame, orient=VERTICAL)
        self.ranklist = Listbox(self.result_frame,yscrollcommand=scrollbar.set)
        self.scrollbar.config(command=self.yview)
        self.scrollbar.pack(side=RIGHT, fill=Y)
        self.ranklist.pack(side=LEFT, fill=BOTH, expand=1)

        root.mainloop()

In [8]:
project = search()

TclError: image "pyimage1" doesn't exist

In [ ]:
    # Create keybox for each keyword
    def create_key():
        self.cur_check = Checkbox(self.key_frame,textvariable=cur_var))
        self.cur_check.place(anchor=CENTER)


    # Delete keybox when clicked x
    def delete_key(Checkbox):
        self.Checkbox.destroy()
        
            def update_search(product,keyword):
            import csv
            key.append(keyword)
            self.cur_var = StringVar()
            self.cur_var.set(keyword)
            #####Need to change output from search_item to csv
            rank_csv =  search_item(product,keyword) 
            for row in rank_csv:
                self.ranklist.insert(row)
            

        # Next Search
        #clear original output
        #update new inputs